# starting from here sep 7

**one solution does't work, lets head to the playground https://platform.openai.com/playground to see how it should work  20mins on langhcian issues**

**all the script for openai https://docs.google.com/document/d/1DSwrFViMLUos7eSvhUKVPptndd1SShLm03WA80PYzxk/edit?usp=sharing**

**Sequential chain model: https://python.langchain.com/docs/modules/chains/foundational/sequential_chains**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import openai
f = open("secret_key.env.txt", "r")
secret_key=f.read()
openai.api_key=secret_key

In [ ]:
df_master = pd.read_csv("df_master_w_significance.csv")

In [ ]:
#pip install langchain

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import SimpleSequentialChain


In [ ]:
#here is a quick start guild
#https://python.langchain.com/docs/get_started/quickstart

# this is for the sequential chain
#https://python.langchain.com/docs/modules/chains/foundational/sequential_chains

In [ ]:
llm = ChatOpenAI(temperature=0, openai_api_key=secret_key)
#defline model and temperature

In [ ]:
template1 = """Here is a list of most popular headlines on reddit:

{headline_content}

First, select at most top 5 most impactful headlines that can 
have impact on the financial market in United States, 
but ignore headlines news about specific companies, 
unless it is banking related with very big macro impact. 
The highest impact headlines are events 
that are highly speculative and out of market expectations. """


In [ ]:
prompt_template = PromptTemplate(input_variables=["headline_content"], template=template1)


In [ ]:
chain1 = LLMChain(llm=llm, prompt=prompt_template, 
                  output_key='summarized_headline') 
#because we have a couple chain
#chain is the model, the prompt and the output


In [ ]:
llm2 = ChatOpenAI(temperature=0,openai_api_key=secret_key)
template2 = """Rate the above headlines sentiment 
from -2 (most negative) to 2 (most positive).

{summarized_headline}

The final output will be a list, 
each value is the sentiment score per headline. 
For example [-1, -2, 0, 1, 2]"""


prompt_template2 = PromptTemplate(input_variables=["summarized_headline"], 
                                  template=template2)

chain2 = LLMChain(llm=llm2, prompt=prompt_template2)




In [ ]:
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

In [ ]:
# lets use a simple example and make it work

In [ ]:
reddit_headlines= """
1 : Mecca under threat: Outrage at plan to destroy the birthplace of the Prophet Mohamed and replace it with a new palace and luxury malls
2 : Russia plans long-range bomber flights near U.S. shores
3 : Northern European nations to boost military cooperation against Russia
4 : Genitalia replaced with ducklings and dolphins in Turkish schoolbooks
5 : Chinese hack U.S. Weather systems, satellite network
6 : 2030 the "far distant future" says Australian Prime Minister Tony Abbott.
7 : Opium poppy cultivation in Afghanistan has reached a record high this year, a UN report has revealed, highlighting the failure of the US-led campaign to crack down on the lucrative crop.
8 : Protesters over missing students in Mexico burn state assembly: Demonstrators set fire to the local legislature building on Wednesday in the capital of the southwestern state of Guerrero in protests over the apparent massacre of 43 students by corrupt police and thugs from drug gangs.
9 : Iraqi Christian fighters retake town
10 : Israel announces it won't cooperate with UN Gaza probe
11 : Alberta fishermen land rare fossil of never-before-seen dinosaur
12 : Russian bomber patrols to reach Gulf of Mexico
13 : Shell ignored internal warnings over Nigeria oil spills, documents suggest | Documents filed at London high court suggest Shell didnt act on advice to fix aged pipeline and underplayed magnitude of two spills in Nigeria in 2008
14 : India wins U.S. support for food scheme, ends WTO blockade
15 : Russia denies NATO accusations over troops in Ukraine
"""

In [ ]:
list_of_sentiment = overall_chain.run(reddit_headlines)

print (list_of_sentiment)


**now that we've learned about sequential chain, lets run it with all the headlines**

In [ ]:
def extract_headline(row_data):
    """
    row_data=df_sig_news.iloc[row_index, :]
    """
    master_string=[str(i+1)+" : "+str(row_data[10:][i]) for i in range(0, len(row_data[10:]))]
    master_string="\n".join(master_string)
    return master_string

In [ ]:
import ast

#first loop it as a function
def sequential_chat_response(overall_chain, headline_content, secret_key):
    """return the average sentiment based on the list of headlines"""
    list_of_sentiment = overall_chain.run(headline_content)
    print ("list of sentiment: ", list_of_sentiment)
    #take the average of the sentiment
    try:
        list_of_sentiment=ast.literal_eval(list_of_sentiment)
        list_of_sentiment=[int(x) for x in list_of_sentiment]
        avg_sentiment=sum(list_of_sentiment)/len(list_of_sentiment)
    except:
        print ("error message not able to find")
        avg_sentiment=0
    return avg_sentiment


In [ ]:
# lets try sentiment round no.2
df_master['sentiment']=0
threshold = 4
for i in range(0,len(df_master)):
    if df_master.loc[i, 'significance']>threshold:
        print ("i is ", i)
        master_string=extract_headline(df_master.iloc[i, 10:35])
        #will get converted to async
        response=sequential_chat_response(overall_chain, master_string, secret_key)
        df_master.loc[i, 'sentiment']=response


**this is quite slow, so lets make it async**

In [ ]:
import asyncio
import time


In [ ]:
#async would pause and until it finish the await function

#first is to get the chat sentiment, lets convert that into async function

In [ ]:
async def get_sequential_chat_response(overall_chain, headline_content, 
                                       secret_key):

    list_of_sentiment = await overall_chain.arun(headline_content)
    print ("list of sentiment: ", list_of_sentiment)
    #take the average of the sentiment
    try:
        list_of_sentiment=ast.literal_eval(list_of_sentiment)
        list_of_sentiment=[int(x) for x in list_of_sentiment]
        avg_sentiment=sum(list_of_sentiment)/len(list_of_sentiment)
    except:
        print ("error message not able to find")
        avg_sentiment=999
    return avg_sentiment


In [ ]:
# second is to conver the loop into async task

In [ ]:
#run it at your own peril, it is quite fast!
async def run_sequential_chat_response(df_master,secret_key, threshold=4):
    df_master['sentiment']=0
    tasks = []
    for i in range(0,len(df_master)):
        if df_master.loc[i, 'significance']>threshold:
            print ("i is ", i)
            master_string=extract_headline(df_master.iloc[i, 10:35]) #top 1 to top 25 columns
            #lets setup each task 
            task = get_sequential_chat_response(overall_chain, master_string, secret_key)
            tasks.append(task)

    # lets use await to call to execute all the task all at once
    results = await asyncio.gather(*tasks) 
    for i, result in enumerate(results):
        df_master.loc[i, 'sentiment'] = result

    return df_master

# lets call the function
df_master= await run_sequential_chat_response(df_master,secret_key)


## technical indicator 

In [ ]:
# lets use technical indicator to enrich this dataset more

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df_master=pd.read_csv("master_sentiment.csv",index_col=0)
df_master.columns


In [ ]:
df_master.rename(columns={
            'Close*': 'Close',
            'Adj Close**': 'Adj Close'},
          inplace=True)
df_master=df_master.reset_index(drop=True)
df_master

In [ ]:
df_master.dropna(inplace=True)
df_master['Date']=pd.to_datetime(df_master['Date'])
df_master

In [ ]:
# lets get rid of all the headline news
df_master = df_master.loc[:, ~df_master.columns.str.contains('Top')]
df_master.head()


In [ ]:
#ma30, ma60, ma120

In [ ]:
df_master["30_SMA"] = df_master["Adj Close"].rolling(window = 30, min_periods = 1).mean()
# create 50 days simple moving average column
df_master["50_SMA"] = df_master["Adj Close"].rolling(window = 50, min_periods = 1).mean()
df_master["200_SMA"] = df_master["Adj Close"].rolling(window = 200, min_periods = 1).mean()


# display first few rows
df_master.head()


In [ ]:
#have a bit issue with the x-axis
plt.plot(df_master['Date'].values, df_master['Adj Close'].values)
plt.plot(df_master['Date'].values, df_master['30_SMA'].values, label = "30_SMA")
plt.plot(df_master['Date'].values, df_master['50_SMA'].values, label = "50_SMA")
plt.plot(df_master['Date'].values, df_master['200_SMA'].values, label = "200_SMA")

plt.legend()
plt.show()




In [ ]:
#Exponential moving average (EMA)

EMAToday=( ValueToday*(Constant/ (1+No. Of Days)) )+( EMAYesterday*(1-(Constant/(1+No. Of Days))) )

In [ ]:
n = 200
#decay = 2/(span+1), min required period..
df_master['EMA'] = df_master['Adj Close'].ewm(ignore_na=False,span=n,min_periods=n,adjust=True).mean()

In [ ]:
plt.plot(df_master['Date'].values, df_master['Adj Close'].values)
plt.plot(df_master['Date'].values, df_master['EMA'].values, label = "ema")
plt.legend()
plt.show()



In [ ]:
#Bollinger Bands

In [ ]:
n = 50
MA = pd.Series(df_master['Adj Close'].rolling(n).mean())
STD = pd.Series(df_master['Adj Close'].rolling(n).std())
bb1 = MA + 2*STD
df_master['Upper Bollinger Band'] = pd.Series(bb1)
bb2 = MA - 2*STD
df_master['Lower Bollinger Band'] = pd.Series(bb2)


In [ ]:
plt.plot(df_master['Date'].values, df_master['Adj Close'].values)
plt.plot(df_master['Date'].values, df_master['Upper Bollinger Band'].values, label = "upper")
plt.plot(df_master['Date'].values, df_master['Lower Bollinger Band'].values, label = "lower")

plt.legend()
plt.show()


#Stochastic oscillator
 The oscillator's value ranges from zero to 100 
 and is typically displayed as two lines: the %K line and the %D line, which is a moving average of %K.
 
 When the oscillator is above 80, the asset is considered overbought. Conversely, a value below 20 indicates an oversold condition. 
 
 comparing a recent closing price to its price range over a specified period
 
 To calculate the %K line, you take the difference between the most recent closing price and the lowest price of the chosen period. You then divide this difference by the difference between the highest and the lowest prices of the same period

In [ ]:
n = 200
smin = df_master['Low'].rolling(n).min()
smax = df_master['High'].rolling(n).max()
df_master['stoch_k'] = 100 * (df_master['Adj Close'] - smin) / (smax - smin)
d_n = 50
df_master['stoch_d'] = df_master['stoch_k'].rolling(d_n).mean()


In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(df_master['Date'].values, df_master['Adj Close'])
ax1.set_title('Closing Price')
ax1.set_ylabel('Price')
ax1.tick_params(axis='x', rotation=45)

ax2 = plt.subplot(2, 1, 2)
ax2.plot(df_master['Date'].values, df_master['stoch_k'], label='Stoch %K')
ax2.plot(df_master['Date'].values, df_master['stoch_d'], label='Stoch %D')
ax2.set_ylabel('Stochastic Oscillator')
ax2.set_xlabel('Date')
ax2.tick_params(axis='x', rotation=45)


#RSI
The RSI is commonly used to identify overbought or oversold conditions in a traded security. An RSI above 70 suggests that an asset may be overbought, indicating it could be a good time to sell. Conversely, an RSI below 30 suggests that the asset may be oversold, which could be a buying opportunity.

RS (Relative Strength) is the average of 'n' days' up closes divided by the average of 'n' days' down closes.
 

In [ ]:
n = 50 # Number of period
change = df_master['Adj Close'].diff(1)
df_master['Gain'] = change.mask(change<0,0)
df_master['Loss'] = abs(change.mask(change>0,0))
df_master['AVG_Gain'] = df_master.Gain.rolling(n).mean()
df_master['AVG_Loss'] = df_master.Loss.rolling(n).mean()
df_master['RS'] = df_master['AVG_Gain']/df_master['AVG_Loss']
df_master['RSI'] = 100 - (100/(1+df_master['RS']))


In [ ]:
#df_master axis is not there
fig = plt.figure(figsize=(14,10))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(df_master['Date'].values, df_master['Adj Close'])
ax1.set_title('Closing Price')
ax1.set_ylabel('Price')

ax2 = plt.subplot(2, 1, 2)
ax2.plot(df_master['RSI'], label='Relative Strengths Index')


#adding the text [30] -- a bit to the left
ax2.text(s='Overbought', x=df_master.RSI.index[30], y=70, fontsize=14)
ax2.text(s='Oversold', x=df_master.RSI.index[30], y=30, fontsize=14)

#adding horizontal axis
ax2.axhline(y=70, color='red')
ax2.axhline(y=30, color='red')
ax2.grid()
ax2.set_ylabel('RSI')
ax2.set_xlabel('Date')


#on balance volumne

The basic idea is that volume often precedes price movement, meaning that the level of trading activity can be an early indicator of future price trends. OBV is used to confirm price trends and identify potential reversals.

The calculation of OBV is straightforward. Starting with an initial value, you add the day's trading volume when the price closes higher than the previous day's close, and subtract the volume when the price closes lower. If the close is the same as the previous day, then the OBV remains unchanged. The formula is as follows:

1. If today's close > yesterday's close, then OBV = Previous OBV + Today's Volume
2. If today's close < yesterday's close, then OBV = Previous OBV - Today's Volume
3. If today's close = yesterday's close, then OBV = Previous OBV

When the OBV is rising, it indicates that volume is stronger on up days, suggesting bullish sentiment. Conversely, a falling OBV suggests that the volume is stronger on down days, which can be an indicator of bearish sentiment.


In [ ]:
import numpy as np
df_master['obv'] = (np.sign(df_master['Adj Close'].diff()) * df_master['Volume']).fillna(0).cumsum()

In [ ]:
fig = plt.figure(figsize=(14,10))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(df_master['Date'].values, df_master['Adj Close'])
ax1.set_title('Closing Price')
ax1.set_ylabel('Price')

ax2 = plt.subplot(2, 1, 2)
ax2.plot(df_master['Date'].values, df_master['obv'], label='On Balance Volume')
ax2.grid()
ax2.set_ylabel('OBV')
ax2.set_xlabel('Date')


#Volume Accumulation Oscillator (VAO)

It aims to identify instances where price moves are supported by volume, 
thereby offering a more comprehensive view of market sentiment.

The VAO is the product of the volume times the difference of the current price and the midpoint price.



In [ ]:
df_master['VAO'] = df_master['Volume'] * \
                   (df_master['Adj Close'] - (df_master['High']+df_master['Low'])/2)
df_master.head()

In [ ]:
df_master['Positive'] = df_master['VAO'] > 0
#df_master['VolumePositive'] = df_master['Open'] < df_master['Adj Close']


In [ ]:
fig = plt.figure(figsize=(14,10))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(df_master['Date'].values, df_master['Adj Close'].values)
ax1.axhline(y=df_master['Adj Close'].mean(),color='r')
ax1.grid()
ax1.set_ylabel('Price')

ax2 = plt.subplot(2, 1, 2)
ax2.bar(df_master['Date'].values, df_master['VAO'].values, color=df_master.Positive.map({True: 'g', False: 'r'}))
ax2.grid()
ax2.set_ylabel('VAO')
ax2.set_xlabel('Date')


In [ ]:
df_master.to_csv("df_master.csv")

## Tree model with enriched data

now that we have all these indicators, our plan is to use a tree model with all these technical indciators

In [ ]:
#https://medium.com/sfu-cspmp/xgboost-a-deep-dive-into-boosting-f06c9c41349
#very good reference

In [ ]:
import pandas as pd
import numpy as np
df_predictions=pd.DataFrame(columns=['Date'])
#df_master=pd.read_csv("df_master.csv", index_col=0)
df_master.head()

In [ ]:
#30_SMA, #50_SMA, #200_SMA, #EMA, #Upper Bollinger Band, #Lower Bollinger Band, #stoch_k, #stoch_d, #RSI, #obv,#VAO
#significance #sentiment => #percentage as the y-axis

In [ ]:
df_master.head()
df_master['percentage_yesterday']=df_master['percentage']

In [ ]:
# lets first shifting all the technical indicator to one lower 
# because they are calculated using the xlosing of today's data

mask = ~(df_master.columns.isin(['Date','Open', 'High', 'Low', 'Close*', 'Adj Close**', 'percentage']))

cols_to_shift = df_master.columns[mask]

df_master[cols_to_shift] = df_master.loc[:,mask].shift(1)


In [ ]:
df_master.head() #look at the positive columsn, we are using yesterday's information

# XGBoost Regression

In [ ]:
import xgboost as xgb

In [ ]:
model=xgb.XGBRegressor()

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X=df_master[['significance', 'sentiment', '30_SMA', '50_SMA', 
             '200_SMA', 'EMA', 'Upper Bollinger Band', 
             'Lower Bollinger Band', 'stoch_k', 'stoch_d', 
             'RSI', 'obv', 'VAO', 'percentage_yesterday']]

In [ ]:
y=df_master[['percentage', 'Date']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 99, shuffle=False)

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
df_predictions['Date']=y_test['Date']

In [ ]:
y_test=y_test.drop(columns='Date')
y_train=y_train.drop(columns='Date')

In [ ]:
X_train.info()

In [ ]:
train_matrix=xgb.DMatrix(data=X_train, label=y_train)
test_matrix=xgb.DMatrix(data=X_test, label=y_test)

In [ ]:
reg_params =  [1, 10, 100]


In [ ]:
params = {"objective": "reg:squarederror"}


In [ ]:
# Create an empty list for storing rmses as a function of ridge regression complexity
ridge_regression = []


In [ ]:
# Iterate over reg_params
for reg in reg_params:

    # Update l2 strength
    params["lambda"] = reg

    # Pass this updated param dictionary into cv, as pandas returning the output as dataframe
    cv_results_rmse = xgb.cv(dtrain=train_matrix, params=params, nfold=5,
                             num_boost_round=5, metrics="rmse", as_pandas=True, seed=123)

    # Append best rmse (final round)
    ridge_regression.append(cv_results_rmse["test-rmse-mean"].tail(1).values[0])



In [ ]:
ridge_regression

In [ ]:
print("Best RMSE as a function of ridge regression (L2 regularization):")
print(pd.DataFrame(list(zip(reg_params, ridge_regression)), columns=["l2", "rmse"]))


In [ ]:
# gets pick the best model
params = {"objective": "reg:squarederror", 
         "lambda": "1"}

bst = xgb.train(dtrain=train_matrix, params=params, num_boost_round=50)




In [ ]:
prediction=bst.predict(test_matrix)

In [ ]:
df_predictions['iterate_prediction']=prediction
df_predictions.head()

In [ ]:
from sklearn.metrics import mean_squared_error
print((mean_squared_error(prediction, y_test, squared=False)))


In [ ]:
xgb.plot_importance(bst)


## Grid Search ##

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
grid_search_params = {
    'colsample_bytree': [0.3, 0.7],
    'learning_rate': [0.01, 0.1, 0.2, 0.5], # the bigger, the more likely to overfit but faster to get the result
    'n_estimators': [100],  # the number of total possible parameter combination crossfold (4) x 2*4*3*3= 288
    'subsample': [0.2, 0.5, 0.8], #fraction to random sample in each training
    'max_depth': [4, 5, 6]
}


In [ ]:
xg_grid_reg = xgb.XGBRegressor(objective= "reg:squarederror")


In [ ]:
grid = GridSearchCV(estimator=xg_grid_reg, param_grid=grid_search_params, 
                    scoring='neg_mean_squared_error',
                    cv=4, verbose=1)


In [ ]:
grid.fit(X_train, y_train)


In [ ]:
grid


In [ ]:
print("Best parameters found: ", grid.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid.best_score_)))



In [ ]:
# gets pick the best model, no need for nestimators
params = {"objective": "reg:squarederror", 
         'colsample_bytree': 0.3, 
          'learning_rate': 0.2, 
          'max_depth': 4, 
      'subsample': 0.2
         }

bst_grid = xgb.train(dtrain=train_matrix, params=params, num_boost_round=50)


In [ ]:
grid_prediction=bst_grid.predict(test_matrix)

In [ ]:
df_predictions['grid_prediction']=grid_prediction

In [ ]:
print((mean_squared_error(grid_prediction, y_test, squared=False)))


In [ ]:
xgb.plot_importance(bst_grid)


## Random Search Parameters


In [ ]:
params_random_search = {
    'learning_rate': np.arange(0.01, 1.01, 0.01),
    'n_estimators': [50, 100, 200],
    'max_depth': range(2, 12),
    'subsample': np.arange(0.02, 1.02, 0.02),
    'colsample_bytree': [0.3, 0.5, 0.7,1],

}


In [ ]:
xg_random_reg = xgb.XGBRegressor(objective= "reg:squarederror")


In [ ]:
# It calculates the mean squared error (MSE) 
#between the predicted and actual values but returns the negative value of it

#However, for error metrics like MSE, lower values are better. 
#By using the negative sign, the algorithm can still aim to maximize the metric, 
#conforming to its default behavior.
randomized_mse = RandomizedSearchCV(param_distributions=params_random_search, estimator=xg_random_reg,
                                scoring="neg_mean_squared_error", n_iter=5, cv=4, verbose=1)


In [ ]:
randomized_mse.fit(X_train, y_train)


In [ ]:
print("Randomize Search Cross Validation")
print("Best parameters found: ", randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))


In [ ]:
# gets pick the best model, no need for nestimators
params = {"objective": "reg:squarederror", 
         'subsample': 0.28,  'max_depth': 7, 'learning_rate': 0.53, 'colsample_bytree': 0.7
         }

#num_boost_round is a parameter in XGBoost, 
#a popular machine learning library for gradient boosting. 
#This parameter specifies the number of boosting rounds or trees to build. 
#Each round adds a new tree to the model, 
#attempting to correct errors made by the existing ensemble of trees.

bst_random = xgb.train(dtrain=train_matrix, params=params, num_boost_round=50)

In [ ]:
random_prediction=bst_random.predict(test_matrix)

In [ ]:
df_predictions['random_prediction']=random_prediction

In [ ]:
print((mean_squared_error(random_prediction, y_test, squared=False)))


In [ ]:
xgb.plot_importance(bst_random)

In [ ]:
df_predictions.to_csv("prediction_output.csv")
df_predictions

# working on backtesting

In [ ]:
#pip install backtesting

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import numpy as np

In [ ]:
df_price = (
    # Load file
    pd.read_csv('dowj_price_indicators.csv', index_col=0)

    # Rename and filter columns
    .rename(columns={'Close*':'Close'})
    [['Date', 'Open', 'High', 'Low', 'Close']]

    # Parse dates, sort by them and set as index
    .assign(Date = lambda df: pd.to_datetime(df.Date))
    .sort_values('Date')
    .set_index('Date', drop=True)
)

In [ ]:
df_price.head()
df_price = df_price.reset_index()
#df_price = df_price.reset_index(drop=True)
df_price['Date'] = pd.to_datetime(df_price['Date'])
df_price

In [ ]:
#only out of sample
df_predictions.head()
df_predictions['Date']=pd.to_datetime(df_predictions['Date'])


In [ ]:
# Predictions includes only testing data so the resulting df has only that as well
df_price_and_predictions = df_predictions.merge(df_price, how='left', on='Date')
df_price_and_predictions=df_price_and_predictions.set_index('Date', drop=True)
df_price_and_predictions

In [ ]:
def identity(x):
    '''Identity, this is hack because we already precomputed'''
    return x

class PredictionStrat(Strategy):
    """
    Open a long position when data.prediction is >= zero (closes short position).
    Open a short position otherwise (closes long position).
    Requires exclusive_orders=True.
    """

    def init(self):
        self.prediction = self.I(identity, self.data.prediction)
        
    def next(self):
        current_prediction = self.prediction[-1]
        if current_prediction >= 0:
            self.buy()
        else:
            self.sell()


In [ ]:
def backtest(df_input: pd.DataFrame, prediction_col: str) -> Backtest:
    """Renames prediction_col to "prediction" and backtests PredictionStrategy with it."""

    # Rename prediction_col to used_col in PredictionStrategy
    _df_input = df_input.rename(columns={prediction_col:'prediction'}, inplace=False)

    # Creates backtest obj, runs and plots
    bt = Backtest(_df_input, PredictionStrat, exclusive_orders=True, cash=100_000_000, commission=0.0001)
    print(bt.run())
    bt.plot()


In [ ]:
backtest(df_price_and_predictions, 'grid_prediction')

In [ ]:
backtest(df_price_and_predictions, 'random_prediction')

In [ ]:
backtest(df_price_and_predictions, 'iterate_prediction')

In [ ]:
# lets try it with bollinger
# we can optimize the trading strategy with backtesting

In [ ]:
class BollingerStrat(Strategy):
    window_len = 50
    n_stds = 2

    def init(self, n_stds=None, window_len=None):
        rolling = self.data.df.Close.rolling(self.window_len)
        upper_bollinger = rolling.mean() + self.n_stds * rolling.std()
        lower_bollinger = rolling.mean() - self.n_stds * rolling.std()
        self.upper_bollinger = self.I(identity, upper_bollinger)
        self.lower_bollinger = self.I(identity, lower_bollinger)
        self.Close = self.I(identity, self.data.Close)

    def next(self):
        if self.Close >= self.upper_bollinger:
            self.buy()
        elif self.Close <= self.lower_bollinger:
            self.sell()

In [ ]:
bt = Backtest(df_price_and_predictions, BollingerStrat, cash=100_000_000, exclusive_orders=True)
print(bt.run())
bt.plot()

In [ ]:
# lets try to optimize for backtesting

In [ ]:
window_lens = list(range(5,105,5))
n_stds = list(np.linspace(0.5, 4, 15))

In [ ]:
from backtesting.lib import plot_heatmaps

bt = Backtest(df_price_and_predictions, BollingerStrat, 
              cash=100_000_000, exclusive_orders=True)

results, heatmap = bt.optimize(window_len=window_lens, 
                               n_stds=n_stds, maximize='Return [%]', return_heatmap=True)

In [ ]:
plot_heatmaps(heatmap)

In [ ]:
results._strategy

In [ ]:
class OptimizedBollingerStrat(BollingerStrat):
    n_stds = 2
    window_len = 10

In [ ]:
bt = Backtest(df_price_and_predictions, OptimizedBollingerStrat, cash=100_000_000, exclusive_orders=True)
print(bt.run())
bt.plot()

In [ ]:
# lets stack all 3 strategies together

In [ ]:
df_price_and_predictions.head()

In [ ]:
def identity(x):
    '''Identity'''
    return x

class SignalStackingStrat1(Strategy):
    """
    Opens a long position whenever 2 or more prediction values
    are positive. Close it otherwise.
    """

    def init(self):
        self.p1 = self.I(identity, self.data.random_prediction)
        self.p2 = self.I(identity, self.data.iterate_prediction)
        self.p3 = self.I(identity, self.data.grid_prediction)

    def next(self):
        # Positive signals
        ps1 = int(self.p1[-1] >= 0)
        ps2 = int(self.p2[-1] >= 0)
        ps3 = int(self.p3[-1] >= 0)

        if (ps1 + ps2 + ps3) >= 2:
            self.buy()
        else:
            self.position.close()

In [ ]:
# Creates backtest obj, runs and plots
bt = Backtest(df_price_and_predictions, SignalStackingStrat1, cash=100_000_000)#, exclusive_orders=True)
print(bt.run())
bt.plot()

In [ ]:
def identity(x):
    '''Identity'''
    return x

class SignalStackingStrat2(Strategy):
    """
    Open a long position whenever the 3 prediction values are positive.
    Open a short position whenever the 3 prediction values are negative.
    """

    def init(self):
        self.p1 = self.I(identity, self.data.random_prediction)
        self.p2 = self.I(identity, self.data.iterate_prediction)
        self.p3 = self.I(identity, self.data.grid_prediction)

    def next(self):
        # Positive signals
        ps1 = int(self.p1[-1] >= 0)
        ps2 = int(self.p2[-1] >= 0)
        ps3 = int(self.p3[-1] >= 0)

        # Negative signals
        ns1 = int(self.p1[-1] < 0)
        ns2 = int(self.p2[-1] < 0)
        ns3 = int(self.p3[-1] < 0)

        if (ps1 + ps2 + ps3) == 3:
            self.buy()
        elif (ns1 + ns2 + ns3) == 3:
            self.sell()

In [ ]:
bt = Backtest(df_price_and_predictions, SignalStackingStrat2, cash=100_000_000, exclusive_orders=True)
print(bt.run())
bt.plot()